In [2]:
import json
from matplotlib import pyplot as plt
from pprint import pprint
import numpy as np
import pandas as pd
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

In [3]:
jdata = json.load(open('data/train/RapidVPData_6918609_693027.json'))
for key in jdata:
    print(key)

EventNumber
MCParticles
MCVertices
RunNumber
VPClusters
VeloTracks


In [4]:
VeloTracks  = jdata['VeloTracks']
VeloHits  = jdata['VPClusters']
mcps = jdata['MCParticles']
mcvs = jdata['MCVertices']

In [6]:
vs =  list(VeloTracks.values())[0]
def getPVParams(track, mcps, mcvs):
    """ Given a track dict, the list of MCParticles and MCVertices 
    return the track PV position and ID (or None if not found or ambiguous)"""
    track_mcps = track['MCPs']
    # We ignore the tracks with no association to MCP or more than one
    if len(track_mcps) != 1:
        return None
    track_mcp = mcps[track_mcps[0]]
    track_pv_id = track_mcp['PV']
    track_pv = mcvs[track_pv_id]
    pos = track_pv['Pos']
    return pos[0], pos[1], pos[2], track_pv['key']

print(vs, getPVParams(vs, mcps, mcvs))

{'ClosestToBeam': [0.01643084393756844, -0.030428914618081393, 21.892009735107422, 0.004823623690754175, -0.010355439968407154, 0.0], 'LHCbIDs': ['2193111806', '2195207147', '2197303265', '2199399639', '2201495757'], 'MCPs': ['8917'], 'errCTBState': [0.006562070954433931, 0.006562070954433931, 2.2658468878944404e-08, 2.2658468878944404e-08, -1.2069097536892594e-05], 'isBackwards': False, 'key': '0'} (-0.0029, 0.0246, 18.7183, '333')
